XceptionNetの実装

In [1]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("GPU Count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


CUDA Available: True
GPU Count: 1
GPU Name: NVIDIA GeForce RTX 4070 Ti SUPER


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import accimage
import os
from PIL import Image
from torchvision import transforms

In [3]:
import random
# 再現性のためにランダムシードを設定
def set_seed(seed):
    random.seed(seed) # Python
    np.random.seed(seed) # NumPy
    torch.manual_seed(seed) # PyTorch
    
    # GPUを使う場合
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)  # 任意のシード値

- acciimageはtorchvisionの画像読み込みバックエンド
- 直接Image()を使って画像を読み込むのではなく、PIL互換のオブジェクトとして扱う必要がある

In [5]:
%%time
image_path = "./data/images"

# 画像の前処理
transform = transforms.Compose([
    #transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

image_files = [f for f in os.listdir(image_path) if f.endswith(('.jpg'))]

image_tensors = []

for file in image_files:
    img = accimage.Image(os.path.join(image_path, file)) # PILで画像を開く
    img = transform(img) # 前処理を適用
    image_tensors.append(img)

image_tensors = torch.stack(image_tensors)

print(f"Loaded {len(image_tensors)} images as tensors.")

: 

In [ ]:
image_tensors.shape

In [ ]:
# PyTorchの CHWフォーマットになっているので、NumPy形式の NHWCフォーマットに変換する
image_tensors = image_tensors.numpy()  # Tensor → NumPy
image_tensors_numpy = np.transpose(image_tensors, (0, 2, 3, 1))  # CHW → HWC (Num_samples, Height, Width, Channels)
print(image_tensors_numpy.shape)

NameError: name 'image_tensors' is not defined

In [ ]:
metadata = pd.read_csv("./data/HAM10000_metadata")
metadata

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern
...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,vidir_modern
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,vidir_modern
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,vidir_modern
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,vidir_modern


In [ ]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lesion_id     10015 non-null  object 
 1   image_id      10015 non-null  object 
 2   dx            10015 non-null  object 
 3   dx_type       10015 non-null  object 
 4   age           9958 non-null   float64
 5   sex           10015 non-null  object 
 6   localization  10015 non-null  object 
 7   dataset       10015 non-null  object 
dtypes: float64(1), object(7)
memory usage: 626.1+ KB


In [ ]:
metadata_labels = metadata[['image_id', 'dx']]
metadata_labels

,image_id,dx
0,ISIC_0027419,bkl
1,ISIC_0025030,bkl
2,ISIC_0026769,bkl
3,ISIC_0025661,bkl
4,ISIC_0031633,bkl
...,...,...
10010,ISIC_0033084,akiec
10011,ISIC_0033550,akiec
10012,ISIC_0033536,akiec
10013,ISIC_0032854,akiec


In [ ]:
labels = metadata_labels['dx'].map({"akiec":0, "bcc":1, "bkl":2, "df":3, "mel":4, "nv":5, "vasc":6})
labels

0        2
1        2
2        2
3        2
4        2
        ..
10010    0
10011    0
10012    0
10013    0
10014    4
Name: dx, Length: 10015, dtype: int64

In [ ]:
labels_encoding = pd.get_dummies(labels, dtype=int)
labels_encoding

,0,1,2,3,4,5,6
0,0,0,1,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0
4,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...
10010,1,0,0,0,0,0,0
10011,1,0,0,0,0,0,0
10012,1,0,0,0,0,0,0
10013,1,0,0,0,0,0,0


In [ ]:
import psutil
print(f"使用中メモリ: {psutil.virtual_memory().used / (1024 ** 3):.2f} GB")

使用中メモリ: 13.97 GB


In [ ]:
# メモリの削減
del image_tensors
import gc
gc.collect()

0

In [ ]:
print(image_tensors_numpy.shape)
print(image_tensors_numpy.nbytes / (1024 ** 3), "GB")  # メモリ使用量（GB単位）

(10015, 224, 224, 3)
5.616016387939453 GB


In [ ]:
from sklearn.model_selection import train_test_split
train_data, val_data, train_labels, val_labels = train_test_split(
    image_tensors_numpy, labels, test_size=0.3, stratify=labels, random_state=42
)

In [ ]:
print("train_data shape:", train_data.shape)
print("train_labels shape:", train_labels.shape)
print("val_data shape:", val_data.shape)
print("val_labels shape:", val_labels.shape)

train_data shape: (7010, 224, 224, 3)
train_labels shape: (7010,)
val_data shape: (3005, 224, 224, 3)
val_labels shape: (3005,)


In [ ]:
class_names_mapping = {
    0: "AKIEC",
    1: "BCC",
    2: "BKL",
    3: "DF",
    4: "MEL",
    5: "NV",
    6: "VASC"
}

In [ ]:
class_counts = np.sum(labels_encoding, axis=0)
class_counts

0     327
1     514
2    1099
3     115
4    1113
5    6705
6     142
dtype: int64

## XceptionNet の事前学習済みモデルを適用

- 学習データの拡張はImageNetとそろえる

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

- このあと、普通に適用としたらエラーになるので、Datasetクラスを作成して適用させる
- torchvision.transforms の ToTensor() は PIL画像 または numpy.ndarray (H, W, C) のみを処理できる

In [ ]:
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform):
        """
        dataはnumpy.darrrayの画像データ（N, H, W, C)
        labelsはnumpy.darrayのラベルデータ（N, ）
        transformは定義した画像変換処理
        """
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = self.data[idx]  # numpy.ndarray (H, W, C)
        label = self.labels[idx]

        # NumPy → PIL に変換（ToTensor() のため）
        image = Image.fromarray((image * 255).astype("uint8"))

        # transform を適用
        if self.transform:
            image = self.transform(image)  # `ToTensor()` により (C, H, W) に変換

        return image, torch.tensor(label, dtype=torch.long)

In [ ]:
train_labels = train_labels.values  # Pandas Series → NumPy 配列
val_labels = val_labels.values      # 検証データも同様に変換

AttributeError: 'numpy.ndarray' object has no attribute 'values'

- torch.long型は整数系（int64）

In [ ]:
batch_size = 32

train_dataset = CustomDataset(train_data, train_labels, transform=train_transform)
val_dataset = CustomDataset(val_data, val_labels, transform=val_transform)

In [ ]:
# CPUのコア数を確認
import os
os.cpu_count()  # コア数

20

In [ ]:
# 訓練用データ
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
# 検証用データ
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
train_loader

In [ ]:
for imgs, labels in train_loader:
    print("Image shape:", imgs.shape)  # 期待する形: (batch_size, C, H, W)
    break

Image shape: torch.Size([16, 3, 224, 224])


In [ ]:
import mlflow
import mlflow.pytorch

# MLflow のトラッキング URI を設定
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# MLflow の実験を設定
mlflow.set_experiment("250311_xceptionnet")

<Experiment: artifact_location='/works/mlruns/1', creation_time=1741694676407, experiment_id='1', last_update_time=1741694676407, lifecycle_stage='active', name='250311_xceptionnet', tags={}>

In [66]:
# モデルを定義
import timm
import torch.nn as nn

num_classes = len(set(train_labels))

def create_model(model_name="legacy_xception", num_classes=num_classes):
    model = timm.create_model(model_name, pretrained=True, num_classes=num_classes)
    return model

In [ ]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    train_loss = 0
    train_correct = 0
    total_samples = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        batch_size = labels.size(0)
        train_loss += loss.item() * batch_size 
        train_correct += (outputs.argmax(1) == labels).sum().item()
        total_samples += batch_size
    
    train_loss /= total_samples
    train_acc = train_correct/ total_samples
    
    return train_loss, train_acc

In [ ]:
def evaluate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0
    val_correct = 0
    total_val_samples = 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            #labels = labels.squeeze().long()  # ラベルを1Dに変換
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            batch_size = labels.size(0)
            val_loss += loss.item() * batch_size
            val_correct += (outputs.argmax(1) == labels).sum().item()
            total_val_samples += batch_size
        
    val_loss /= total_val_samples
    val_acc = val_correct / total_val_samples
        
    return val_loss, val_acc,

In [ ]:
# モデルの作成・訓練・評価・ログ記録
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

def objective(train_loader, val_loader, num_classes, num_epochs):
    set_seed(42)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    model = create_model(num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer =optim.Adam(model.parameters(), lr=0.0001)
    scheduler = StepLR(optimizer, step_size=5, gamma=0.5)

    best_val_loss = float("inf")
    best_epoch = 0
    best_acc = 0
    best_model_state = None

    with mlflow.start_run() as run:
        mlflow.log_param("learning_rate", 0.0001)

        for epoch in range(num_epochs):
            train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
            val_loss, val_acc = evaluate(model, val_loader, criterion, device)

            mlflow.log_metric("train_loss", train_loss, step=epoch)
            mlflow.log_metric("train_acc", train_acc, step=epoch)
            mlflow.log_metric("val_loss", val_loss, step=epoch)
            mlflow.log_metric("val_acc", val_acc, step=epoch)

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_epoch = epoch
                best_acc = val_acc
                best_model_state = model.state_dict()
                mlflow.pytorch.log_model(model, "best_model")
                
            scheduler.step()

            print(f"Epoch {epoch}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        return best_epoch, best_val_loss, best_acc, best_model_state            


In [ ]:
num_epochs = 10
best_epoch, best_val_loss, best_acc, best_model_state = objective(train_loader, val_loader, num_classes, num_epochs)

print("Best Epoch:", best_epoch)
print("Best Val Loss:", best_val_loss)
print("Best Acc:", best_acc)
print("Best Model Sate:", best_model_state)

Epoch 0: Train Loss: nan, Train Acc: 0.0327, Val Loss: nan, Val Acc: 0.0326
Epoch 1: Train Loss: nan, Train Acc: 0.0327, Val Loss: nan, Val Acc: 0.0326
Epoch 2: Train Loss: nan, Train Acc: 0.0327, Val Loss: nan, Val Acc: 0.0326
Epoch 3: Train Loss: nan, Train Acc: 0.0327, Val Loss: nan, Val Acc: 0.0326
Epoch 4: Train Loss: nan, Train Acc: 0.0327, Val Loss: nan, Val Acc: 0.0326
Epoch 5: Train Loss: nan, Train Acc: 0.0327, Val Loss: nan, Val Acc: 0.0326
Epoch 6: Train Loss: nan, Train Acc: 0.0327, Val Loss: nan, Val Acc: 0.0326
Epoch 7: Train Loss: nan, Train Acc: 0.0327, Val Loss: nan, Val Acc: 0.0326
🏃 View run rogue-roo-650 at: http://127.0.0.1:5000/#/experiments/1/runs/5daf8e5fc9544ab3ad91230827bbbcba
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


KeyboardInterrupt: 